# Example of HTML capabilities of project inside IPython notebook

## Initializing Environment

In [1]:
from openerp_proxy.session import IPYSession as Session # Use IPython-itegrated session class
from openerp_proxy.ext.all import HField
import openerp_proxy.plugins.module_utils  # Enable module_utils plugin

# Create session to use. 
# Usualy use `Session()` which will automaticaly use
# '~/.openerp_proxy.json' session file
session = Session('~/.openerp_proxy.local.json')
session

DB URL,DB Index,DB Aliases
json-rpc://admin@localhost:8069/oev8-test1,1,
xml-rpc://admin@localhost:8169/oev7-test4,2,ldb


Table above describes list of connection settings stored in session file and what to do to connect to one
of connections above

## Connect to odoo database by aliase

In [2]:
ldb = session.ldb
ldb

Host,localhost
Port,8169
Protocol,xml-rpc
Database,oev7-test4
login,admin


## Get object / model

*ldb* here represents database connection (*ERP_Proxy* class instance)
As told in help message above *.get_obj* method allows to get instance of specified
*Object* proxy, where Object means *model*, *document*.

In [3]:
so = ldb.get_obj('sale.order')
so

Name,Sales Order
Proxy,xml-rpc://admin@localhost:8169/oev7-test4
Model,sale.order
Record count,11


Also it is posible to use shorter (dictionary style) syntax:

In [4]:
so = ldb['sale.order']
so

Name,Sales Order
Proxy,xml-rpc://admin@localhost:8169/oev7-test4
Model,sale.order
Record count,11


And as result of using [*openerp_proxy.ext.sugar*](http://pythonhosted.org/openerp_proxy/module_ref/openerp_proxy.ext.html#module-openerp_proxy.ext.sugar) extension
(which is automaticaly imported in 'openerp_proxy.ext.all')
there are attribute-style access (which also support's IPython auto-completition):

In [5]:
so = ldb._sale_order
so

Name,Sales Order
Proxy,xml-rpc://admin@localhost:8169/oev7-test4
Model,sale.order
Record count,11


## Getting information about available columns for Object

In [6]:
so.columns_info

name,info.string,info.type,info.selection,info.help,info.readonly,info.required,info.relation_field,info.select
amount_tax,Taxes,float,,The tax amount.,1,None,None,None
amount_total,Total,float,,The total amount.,1,None,None,None
amount_untaxed,Untaxed Amount,float,,The amount without tax.,1,None,None,None
carrier_id,Delivery Method,many2one,,Complete this field if you plan to invoice the shipping based on picking.,None,None,None,None
categ_ids,Categories,many2many,,None,None,None,None,None
client_order_ref,Customer Reference,char,,None,None,None,None,None
company_id,Company,many2one,,None,True,None,None,None
create_date,Creation Date,datetime,,Date on which sales order is created.,True,None,None,True
currency_id,Currency,many2one,,None,True,True,None,None
date_confirm,Confirmation Date,date,,Date on which sales order is confirmed.,True,None,None,True


## Search for sale orders

In [7]:
# Standard search .search_records(domain)
so_list = so.search_records([])
so_list

Object,Object ('sale.order')
Proxy,xml-rpc://admin@localhost:8169/oev7-test4
Record count,11


Also there are shorter syntax provided by [*openerp_proxy.ext.sugar*](http://pythonhosted.org/openerp_proxy/module_ref/openerp_proxy.ext.html#module-openerp_proxy.ext.sugar) extension:

In [8]:
so_list = so([])
so_list

Object,Object ('sale.order')
Proxy,xml-rpc://admin@localhost:8169/oev7-test4
Record count,11


## Display Sale orders as HTML table

It is posible to represent RecordList as HTML table, with ability to highlight rows by specified conditions.
Also, when building result table, it is posible to display values of related fields, and even method calls. This functionality is implemented in [*openerp_proxy.ext.repr*](http://pythonhosted.org/openerp_proxy/module_ref/openerp_proxy.ext.html#module-openerp_proxy.ext.repr) module

In [9]:
# High light rows by condition
highlighters = {
    '#99FF99': lambda x: x.state == 'done',
    '#9999FF': lambda x: x.state == 'draft',
    '#FFFF99': lambda x: x.state == 'progress',
}

# Display as table.
# Note that prefetch method is used to fetch some set of fields via one RPC call.
# on big datasets it may speed up performance signifiantly.
# Each RecordList instance have related cache, which reduce need of reading data on each field get.
so_list.prefetch('id', 'name', 'partner_id', 'state')
so_table = so_list.as_html_table('id',
                                 'name',
                                 # _name attribute provides result of *name_search method:
                                 HField('partner_id._name', name='Partner name'),
                                 # Also it is posible to display result of method calls
                                 # 'as_html_list()' is method of RecordList.
                                 'order_line.as_html_list',
                                 # Argument can be passed as tuple, (field, field name)
                                 ('invoice_ids.as_html_list', 'Related Invoices'),
                                 'state',
                                 highlighters=highlighters,
)
so_table

id,name,Partner name,order_line.as_html_list,Related Invoices,state
11,SO011,Administrator,11: [x-product-13] xproduct-13,,manual
10,SO010,Customer 1,10: [product1] Product1,30: Повернення P-CARGO-0001529: Повернення постачальнику P-CARGO-00015,manual
9,SO009,Customer 1,9: Product 2,24: Інвойс,progress
8,SO008,Customer 1,8: Product 2,25: Інвойс P-CARGO-0000723: Інвойс,progress
7,SO007,Customer 1,7: Product1,38: Повернення постачальнику P-CARGO-0000826: Повернення постачальнику P-CARGO-0000819: Повернення постачальнику P-CARGO-0000818: Повернення постачальнику P-CARGO-0000816: Інвойс P-CARGO-00006,manual
6,SO006,Customer 1,6: Product1,37: Інвойс P-CARGO-0000536: Інвойс P-CARGO-0000535: Інвойс P-CARGO-0000534: Інвойс P-CARGO-0000515: Інвойс P-CARGO-00005,manual
5,SO005,Customer 1,5: Product1,12: SAJ/2014/0002 P-CARGO-00004,manual
4,SO004,Customer 1,4: Product1,9: Інвойс P-CARGO-000036: Інвойс P-CARGO-00002,manual
3,SO003,Customer 1,3: Product1,28: Інвойс постачальника P-CARGO-0001427: Інвойс P-CARGO-00014,manual
2,SO002,Customer 1,2: Product1,1: SAJ/2014/0001,done


There also available to_csv method, which allow to represent table in csv format

In [10]:
so_table.to_csv()

/home/katyukha/projects/erp-proxy/examples/tmp/tmpugbsV8.csv

## Access one element of recordlist via index

In [11]:
so_list[0]


Object,Object ('sale.order')
Proxy,xml-rpc://admin@localhost:8169/oev7-test4
Name,SO011


## Display one sale order as HTML Table

In [12]:
so_list[0].as_html('name',
                   'origin',
                   'partner_id',        # Will display Reacord instance representing partner related to this sale order
                   'partner_id._name',  # Will display result of 'name_get' called on partner
                   'partner_id.sale_order_ids.length')  # Display how many sale orders have this partner

Column,Value
name,SO011
origin,False
partner_id,"R(res.partner, 3)[Administrator]"
partner_id._name,Administrator
partner_id.sale_order_ids.length,1


In [13]:
so_list[0].as_html()  # Display all fields for firest sale order record

Column,Value
Categories,RecordList(crm.case.categ): length=0
Confirmation Date,2014-08-08
Contract / Analytic,False
Create Invoice,manual
Creation Date,2014-08-08 15:05:43
Customer,"R(res.partner, 3)[Administrator]"
Customer Reference,False
Date,2014-08-08
Delivered,False
Delivery Address,"R(res.partner, 3)[Administrator]"
